# Классификация на FashionMNIST (CNN)

In [1]:
import torch
import numpy as np
import torchvision as tv
import time
import torch.nn as nn

In [2]:
BATCH_SIZE=256

In [3]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)

cuda:0


In [4]:
train_dataset = tv.datasets.FashionMNIST('.', train=True, transform=tv.transforms.ToTensor(), download=True)
test_dataset = tv.datasets.FashionMNIST('.', train=False, transform=tv.transforms.ToTensor(), download=True)
train = torch.utils.data.DataLoader(train_dataset, batch_size=BATCH_SIZE)
test = torch.utils.data.DataLoader(test_dataset, batch_size=BATCH_SIZE)
train_dataset.classes

['T-shirt/top',
 'Trouser',
 'Pullover',
 'Dress',
 'Coat',
 'Sandal',
 'Shirt',
 'Sneaker',
 'Bag',
 'Ankle boot']

In [9]:
train_dataset[0][0].shape

torch.Size([1, 28, 28])

In [5]:
model = nn.Sequential(nn.Conv2d(in_channels=1, out_channels=32, kernel_size=3, padding=1),
                      nn.BatchNorm2d(32),
                      nn.ReLU(),
                      nn.MaxPool2d(kernel_size=2, stride=2),
                      nn.Conv2d(in_channels=32, out_channels=64, kernel_size=3),
                      nn.BatchNorm2d(64),
                      nn.ReLU(),
                      nn.MaxPool2d(2),
                      nn.Flatten(),
                      nn.Linear(in_features=2304, out_features=120),
                      nn.BatchNorm1d(120),nn.Dropout(), nn.ReLU(),
                      nn.Linear(in_features=120, out_features=84),
                      nn.BatchNorm1d(84), nn.Dropout(.7), 
                      nn.Linear(in_features=84, out_features=10)
                      )
model = model.to(device)

In [7]:
loss_f = nn.CrossEntropyLoss()
trainer_1 = torch.optim.AdamW(model.parameters(), lr=.01)
trainer_2 = torch.optim.SGD(model.parameters(), lr=.001)
num_epochs = 15

In [8]:
def training(X, y, model, dict, trainer=False):
    X, y = X.to(device), y.to(device)
    if trainer:
        trainer.zero_grad()
    predictions = model(X)
    loss = loss_f(predictions, y)
    loss.backward()
    if trainer:
        trainer.step()
    dict = {'loss':(dict['loss']+loss.item()), 
            'tp':(dict['tp']+(predictions.argmax(dim=1) == y).sum().item()), 
            'iters':(dict['iters']+1), 
            'len':(dict['len']+len(X))}
    return dict    

In [9]:
print('|{: ^8}|{: ^9}|{: ^17}|{: ^16}|{: ^11}|{: ^16}|{: ^15}|{: ^10}|'\
      .format('Epochs','Time','Train Adam loss','Train SGD loss',
              'Test loss','Train Adam acc','Train SGD acc','Test acc'))
for epoch in range(num_epochs):
    start=time.time()
    model.train()
    first_train = {'loss':0, 'tp':0, 'iters':0, 'len':0}
    for X,y in train:
        first_train = training(X, y, model, first_train, trainer_1)
        
    second_train = {'loss':0, 'tp':0, 'iters':0, 'len':0}
    for X,y in train:
        second_train = training(X, y, model, second_train, trainer_2)
    
    model.eval()
    last_train = {'loss':0, 'tp':0, 'iters':0, 'len':0}
    for X,y in test:
        last_train = training(X, y, model, last_train, )
        
    print('|{: ^8}|{: ^9.4f}|{: ^17.4f}|{: ^16.4f}|{: ^11.4f}|{: ^16.4%}|{: ^15.4%}|{: ^10.4%}|'\
          .format(epoch, time.time()-start,  
                  first_train['loss']/first_train['iters'],
                  second_train['loss']/second_train['iters'], 
                  last_train['loss']/last_train['iters'],
                  first_train['tp']/first_train['len'],
                  second_train['tp']/second_train['len'],
                  last_train['tp']/last_train['len']))

| Epochs |  Time   | Train Adam loss | Train SGD loss | Test loss | Train Adam acc | Train SGD acc | Test acc |
|   0    | 68.7784 |     0.5664      |     0.4047     |  0.3307   |    79.9367%    |   86.0683%    | 88.7000% |
|   1    | 68.5101 |     0.3828      |     0.3424     |  0.2909   |    87.0217%    |   88.3817%    | 89.4700% |
|   2    | 68.0033 |     0.3288      |     0.2940     |  0.2621   |    88.8317%    |   89.8000%    | 90.3900% |
|   3    | 68.0298 |     0.2985      |     0.2729     |  0.2528   |    89.8817%    |   90.6750%    | 90.8300% |
|   4    | 67.9169 |     0.2822      |     0.2633     |  0.2513   |    90.2700%    |   90.9700%    | 91.0600% |
|   5    | 67.4403 |     0.2628      |     0.2374     |  0.2359   |    90.9600%    |   91.8267%    | 91.5100% |
|   6    | 67.4809 |     0.2492      |     0.2414     |  0.2404   |    91.3417%    |   91.7533%    | 91.6100% |
|   7    | 68.7975 |     0.2379      |     0.2271     |  0.2396   |    91.7133%    |   92.0650%    | 91.

In [6]:
class NetPipeline:
    def __init__(self, train_loader, test_loader, model, num_epochs, 
                 first_opt, lr1, criterion=nn.CrossEntropyLoss(), second_opt=False, lr2=False):
        self.train_loader = train_loader
        self.test_loader = test_loader
        self.model = model
        self.num_epochs = num_epochs
        self.criterion = criterion
        self.first_opt = first_opt
        self.lr1 = lr1
        self.second_opt = second_opt
        self.lr2 = lr2
        
    def train_steps(self):
        trainer_1 = self.first_opt((self.model).parameters(), lr=self.lr1)
        if self.second_opt: trainer_2 = self.second_opt(self.model.parameters(), lr=self.lr2)
        for epoch in range(self.num_epochs):
            start = time.time()
            self.first_train = self.training(self.train_loader, trainer=trainer_1)
            self.second_train = self.training(self.train_loader, trainer=trainer_2) if self.second_opt else False 
            self.last_train = self.training(self.test_loader, mode='eval')
            time_spent = time.time() - start
            self.printer(epoch, time_spent)
            
    
    def training(self, data, trainer=False, mode='train'):
        self.model.eval() if mode == 'eval' else self.model.train()
        dict = {'loss':0, 'tp':0, 'iters':0, 'len':0}
        for X, y in data:
            X, y = X.to(device), y.to(device)
            if mode == 'train': trainer.zero_grad()
            predictions = self.model(X)
            loss = self.criterion(predictions, y)
            loss.backward()
            if mode == 'train': trainer.step()
            dict = {'loss':(dict['loss']+loss.item()), 
                    'tp':(dict['tp']+(predictions.argmax(dim=1) == y).sum().item()), 
                    'iters':(dict['iters']+1), 
                    'len':(dict['len']+len(X))}
        return {'loss':(dict['loss']/dict['iters']), 'acc':(dict['tp']/dict['len'])} 
            
    
    def printer (self, epoch, time_spent):
        name_1 = self.first_opt.__name__
        name_2 = self.second_opt.__name__ if self.second_opt else False
        if not epoch:
            print('|{: ^8}|{: ^9}|'.format('Epochs','Time') + \
                  '{: ^17}|{: ^16}|'.format('Train '+name_1+' loss', 'Train '+name_1+' acc') + \
                  ('{: ^17}|{: ^16}|'.format('Train '+str(name_2)+' loss', 
                                             'Train '+str(name_2)+' acc')if self.second_opt else '') + \
                  '{: ^11}|{: ^10}|'.format('Test loss', 'Test acc'))

        print('|{: ^8}|{: ^9.4f}|'.format(epoch, time_spent) + \
              '{: ^17.4f}|{: ^16.4%}|'.format(self.first_train['loss'], self.first_train['acc']) + \
              ('{: ^17.4f}|{: ^16.4%}|'.format(self.second_train['loss'], 
                                               self.second_train['acc'])if self.second_opt else '') + \
              '{: ^11.4f}|{: ^10.4%}|'.format(self.last_train['loss'], self.last_train['acc']))

In [8]:
NetPipeline(train, test, model, 5, first_opt=torch.optim.AdamW, lr1=.01, second_opt=torch.optim.SGD, lr2=.001 ).train_steps()

| Epochs |  Time   |Train AdamW loss |Train AdamW acc | Train SGD loss  | Train SGD acc  | Test loss | Test acc |
|   0    | 78.8442 |     0.5544      |    80.7500%    |     0.3859      |    86.9350%    |  0.3104   | 88.9400% |
|   1    | 72.5930 |     0.3719      |    87.4317%    |     0.3241      |    88.9500%    |  0.2743   | 89.9200% |
|   2    | 70.5551 |     0.3242      |    88.9317%    |     0.2947      |    90.1083%    |  0.2560   | 90.6600% |
|   3    | 68.9690 |     0.2970      |    89.8350%    |     0.2677      |    90.9050%    |  0.2468   | 91.1300% |


KeyboardInterrupt: 